In [11]:
import pandas as pd
import numpy as np
from scipy.signal import medfilt
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [12]:
METADATA = {
    'oskar': {
        'person_id': '001',  # TODO: Fill in your assigned ID
        'sex': 1,  # 0 = weiblich, 1 = männlich
        'age': 26,  # TODO: Fill in
        'exercise': 'running',  # running, stairs, lunges, mountain_climbers
        'diet': 0,  # 0 = Fleisch, 1 = Vegetarisch, 2 = Vegan
        'caffeine': 1,  # 0 = nein, 1 = ja
        'sleep_q': 2,  # 0 = schlecht, 1 = mittel, 2 = gut
        'sleep_h': 7,  # Schlafdauer in Stunden
        'wellbeeing': 20,  # WHO-5 Fragebogen (0-25 Punkte)
        'cold_hist': 0,  # 0 = nein, 1 = ja
        'asthma': 0,  # 0 = nein, 1 = ja
        'smoker': 1,  # 0 = nein, 1 = ja
        'fitness_level': 0,  # 0 = low, 1 = medium, 2 = high
        'study_modus': 1,  # 0 = vollzeit, 1 = teilzeit
        'bmi': 21,  # Body-Mass-Index
        'rpm': 0,  # Wiederholungen pro Minute (Kadenz)
    },
    'michelle': {
        'person_id': '002',  # TODO: Fill in your assigned ID
        'sex': 0,  # 0 = weiblich, 1 = männlich
        'age': 22,  # TODO: Fill in
        'exercise': 'running',  # running, stairs, lunges, mountain_climbers
        'diet': 0,  # 0 = Fleisch, 1 = Vegetarisch, 2 = Vegan
        'caffeine': 0,  # 0 = nein, 1 = ja
        'sleep_q': 0,  # 0 = schlecht, 1 = mittel, 2 = gut
        'sleep_h': 0.5,  # Schlafdauer in Stunden
        'wellbeeing': 11,  # WHO-5 Fragebogen (0-25 Punkte)
        'cold_hist': 0,  # 0 = nein, 1 = ja
        'asthma': 0,  # 0 = nein, 1 = ja
        'smoker': 0,  # 0 = nein, 1 = ja
        'fitness_level': 1,  # 0 = low, 1 = medium, 2 = high
        'study_modus': 0,  # 0 = vollzeit, 1 = teilzeit
        'bmi': 0.0,  # Body-Mass-Index
        'rpm': 0,  # Wiederholungen pro Minute (Kadenz)
    },
    'eros': {
        'person_id': '003',  # TODO: Fill in your assigned ID
        'sex': 1,  # 0 = weiblich, 1 = männlich
        'age': 26,  # TODO: Fill in
        'exercise': 'running',  # running, stairs, lunges, mountain_climbers
        'diet': 0,  # 0 = Fleisch, 1 = Vegetarisch, 2 = Vegan
        'caffeine': 1,  # 0 = nein, 1 = ja
        'sleep_q': 2,  # 0 = schlecht, 1 = mittel, 2 = gut
        'sleep_h': 8,  # Schlafdauer in Stunden
        'wellbeeing': 20,  # WHO-5 Fragebogen (0-25 Punkte)
        'cold_hist': 0,  # 0 = nein, 1 = ja
        'asthma': 1,  # 0 = nein, 1 = ja
        'smoker': 0,  # 0 = nein, 1 = ja
        'fitness_level': 1,  # 0 = low, 1 = medium, 2 = high
        'study_modus': 0,  # 0 = vollzeit, 1 = teilzeit
        'bmi': 26,  # Body-Mass-Index
        'rpm': 0,  # Wiederholungen pro Minute (Kadenz)
    }
}



ZHAW_KUERZEL = {'oskar': 'steinosk', 'michelle': 'linamic', 'eros': 'halero01'}


In [13]:
def process_person(person, zhaw_id):
    # Only process raw files (not cleaned ones)
    raw_files = sorted(Path("data").glob(f"{zhaw_id}_activity_intensity_[1-3]_lsdlpr25.csv"))
    comparisons = []
    
    for raw_file in raw_files:
        intensity = int(raw_file.stem.split('_')[3])
        df = pd.read_csv(raw_file)
        
        bpm_clean = clean_bpm(df['bpm'].values)
        artifacts = np.sum(df['bpm'].values != bpm_clean)
        
        row = {'time': df['time'], 'rr_interval': df['rr_interval'], 'bpm_raw': df['bpm'], 'bpm_clean': bpm_clean, 'intensity': intensity, 'artifacts': artifacts}
        for k, v in METADATA[person].items():
            row[k] = v
        
        df_clean = pd.DataFrame(row)
        df_clean.to_csv(f"data/{zhaw_id}_activity_intensity_{intensity}_cleaned_lsdlpr25.csv", index=False)
        
        comparisons.append({'intensity': intensity, 'raw': df['bpm'].values, 'clean': bpm_clean, 'mean_raw': df['bpm'].mean(), 'mean_clean': bpm_clean.mean(), 'std_raw': df['bpm'].std(), 'std_clean': bpm_clean.std()})
        print(f"✅ {zhaw_id}_activity_intensity_{intensity}_cleaned_lsdlpr25.csv (artifacts: {artifacts})")
    
    return comparisons

In [14]:
def clean_bpm(bpm, window=5):
    cleaned = medfilt(bpm, kernel_size=window)
    cleaned = np.where((cleaned < 40) | (cleaned > 200), np.median(cleaned), cleaned)
    return cleaned
        # Special handling for Eros




def process_person(person, zhaw_id):
    # Only process raw files (not cleaned ones)
    raw_files = sorted(Path("data").glob(f"{zhaw_id}_activity_intensity_[1-3]_lsdlpr25.csv"))
    comparisons = []
    
    for raw_file in raw_files:
        intensity = int(raw_file.stem.split('_')[3])
        df = pd.read_csv(raw_file)
        
        bpm_clean = clean_bpm(df['bpm'].values)
        artifacts = np.sum(df['bpm'].values != bpm_clean)
        
        row = {'time': df['time'], 'rr_interval': df['rr_interval'], 'bpm_raw': df['bpm'], 'bpm_clean': bpm_clean, 'intensity': intensity, 'artifacts': artifacts}
        for k, v in METADATA[person].items():
            row[k] = v
        
        df_clean = pd.DataFrame(row)
        df_clean.to_csv(f"data/{zhaw_id}_activity_intensity_{intensity}_cleaned_lsdlpr25.csv", index=False)
        
        comparisons.append({'intensity': intensity, 'raw': df['bpm'].values, 'clean': bpm_clean, 'mean_raw': df['bpm'].mean(), 'mean_clean': bpm_clean.mean(), 'std_raw': df['bpm'].std(), 'std_clean': bpm_clean.std()})
        print(f"✅ {zhaw_id}_activity_intensity_{intensity}_cleaned_lsdlpr25.csv (artifacts: {artifacts})")
    
    return comparisons

In [15]:
def plot_comparison(person, zhaw_id, data):
    fig = make_subplots(rows=2, cols=3, subplot_titles=[f"Intensity {d['intensity']}" for d in data] + [f"Stats {d['intensity']}" for d in data], specs=[[{'type': 'scatter'}, {'type': 'scatter'}, {'type': 'scatter'}], [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]])
    
    for i, d in enumerate(data, 1):
        fig.add_trace(go.Scatter(y=d['raw'], name=f"Raw {d['intensity']}", opacity=0.6, line=dict(color='red')), row=1, col=i)
        fig.add_trace(go.Scatter(y=d['clean'], name=f"Clean {d['intensity']}", line=dict(color='green')), row=1, col=i)
        fig.add_trace(go.Bar(x=['Mean', 'Std'], y=[d['mean_raw'], d['std_raw']], name=f"Raw {d['intensity']}", marker=dict(color='red'), opacity=0.6), row=2, col=i)
        fig.add_trace(go.Bar(x=['Mean', 'Std'], y=[d['mean_clean'], d['std_clean']], name=f"Clean {d['intensity']}", marker=dict(color='green')), row=2, col=i)
    
    fig.update_layout(height=800, title_text=f"{person.capitalize()} - Raw vs Cleaned BPM", showlegend=False)
    fig.show()

print("=== Datenaufbereitung & Vergleich ===\n")
for person, zhaw_id in ZHAW_KUERZEL.items():
    print(f"Processing {person.capitalize()}...")
    data = process_person(person, zhaw_id)
    plot_comparison(person, zhaw_id, data)
    print()

print("✅ Datenaufbereitung abgeschlossen!")


=== Datenaufbereitung & Vergleich ===

Processing Oskar...
✅ steinosk_activity_intensity_1_cleaned_lsdlpr25.csv (artifacts: 231)
✅ steinosk_activity_intensity_2_cleaned_lsdlpr25.csv (artifacts: 241)
✅ steinosk_activity_intensity_3_cleaned_lsdlpr25.csv (artifacts: 291)



Processing Michelle...
✅ linamic_activity_intensity_1_cleaned_lsdlpr25.csv (artifacts: 224)
✅ linamic_activity_intensity_2_cleaned_lsdlpr25.csv (artifacts: 240)
✅ linamic_activity_intensity_3_cleaned_lsdlpr25.csv (artifacts: 288)



Processing Eros...
✅ halero01_activity_intensity_1_cleaned_lsdlpr25.csv (artifacts: 163)
✅ halero01_activity_intensity_2_cleaned_lsdlpr25.csv (artifacts: 201)
✅ halero01_activity_intensity_3_cleaned_lsdlpr25.csv (artifacts: 280)



✅ Datenaufbereitung abgeschlossen!


In [16]:
# Final Combined Dataset for Submission
import pandas as pd
from pathlib import Path

# Combine all cleaned datasets into final submission format
cleaned_files = Path("data").glob("*_cleaned_lsdlpr25.csv")

# show head of each cleaned file
for file in sorted(cleaned_files):
    df = pd.read_csv(file)
    display(df.describe())


,rr_interval,bpm_raw,bpm_clean,intensity,artifacts,person_id,sex,age,diet,caffeine,sleep_q,sleep_h,wellbeeing,cold_hist,asthma,smoker,fitness_level,study_modus,bmi,rpm
count,252.000000,252.000000,252.000000,252.0,252.0,252.0,252.0,252.0,252.0,252.0,252.0,252.0,252.0,252.0,252.0,252.0,252.0,252.0,252.0,252.0
mean,477.595238,128.181052,128.244736,1.0,163.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
std,88.672654,15.439102,13.836624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,352.000000,40.160643,81.411126,1.0,163.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
25%,428.000000,117.187500,117.187500,1.0,163.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
50%,458.500000,130.861661,130.718954,1.0,163.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
75%,512.000000,140.186916,140.186916,1.0,163.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
max,1494.000000,170.454545,145.278450,1.0,163.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0


,rr_interval,bpm_raw,bpm_clean,intensity,artifacts,person_id,sex,age,diet,caffeine,sleep_q,sleep_h,wellbeeing,cold_hist,asthma,smoker,fitness_level,study_modus,bmi,rpm
count,288.000000,288.000000,288.000000,288.0,288.0,288.0,288.0,288.0,288.0,288.0,288.0,288.0,288.0,288.0,288.0,288.0,288.0,288.0,288.0,288.0
mean,528.121528,139.862906,147.963149,2.0,201.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
std,463.928741,41.081449,26.395088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,253.000000,11.525163,68.181818,2.0,201.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
25%,364.000000,123.456790,144.930061,2.0,201.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
50%,387.500000,154.838967,156.250000,2.0,201.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
75%,486.000000,164.835165,164.383562,2.0,201.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
max,5206.000000,237.154150,195.439739,2.0,201.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0


,rr_interval,bpm_raw,bpm_clean,intensity,artifacts,person_id,sex,age,diet,caffeine,sleep_q,sleep_h,wellbeeing,cold_hist,asthma,smoker,fitness_level,study_modus,bmi,rpm
count,371.000000,371.000000,371.000000,371.0,371.0,371.0,371.0,371.0,371.0,371.0,371.0,371.0,371.0,371.0,371.0,371.0,371.0,371.0,371.0,371.0
mean,406.843666,164.121584,164.522206,3.0,280.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
std,286.436319,32.205375,28.042127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,272.000000,12.950572,47.732697,3.0,280.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
25%,335.000000,171.428571,171.428571,3.0,280.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
50%,343.000000,174.927114,174.418605,3.0,280.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
75%,350.000000,179.104478,179.104478,3.0,280.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0
max,4633.000000,220.588235,181.818182,3.0,280.0,3.0,1.0,26.0,0.0,1.0,2.0,8.0,20.0,0.0,1.0,0.0,1.0,0.0,26.0,0.0


,rr_interval,bpm_raw,bpm_clean,intensity,artifacts,person_id,sex,age,diet,caffeine,sleep_q,sleep_h,wellbeeing,cold_hist,asthma,smoker,fitness_level,study_modus,bmi,rpm
count,362.000000,362.000000,362.000000,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0
mean,424.162983,141.569808,141.580240,1.0,224.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
std,12.213632,3.998819,3.907033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,403.000000,129.310345,130.718954,1.0,224.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25%,415.000000,139.292074,139.534884,1.0,224.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
50%,423.000000,141.843972,141.843972,1.0,224.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
75%,430.750000,144.578313,144.578313,1.0,224.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
max,464.000000,148.883375,148.148148,1.0,224.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


,rr_interval,bpm_raw,bpm_clean,intensity,artifacts,person_id,sex,age,diet,caffeine,sleep_q,sleep_h,wellbeeing,cold_hist,asthma,smoker,fitness_level,study_modus,bmi,rpm
count,395.000000,395.000000,395.000000,395.0,395.0,395.0,395.0,395.0,395.0,395.0,395.0,395.0,395.0,395.0,395.0,395.0,395.0,395.0,395.0,395.0
mean,387.559494,156.960702,157.915432,2.0,240.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
std,56.354330,15.560534,12.735487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,347.000000,66.815145,114.285714,2.0,240.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25%,356.000000,150.000000,150.564854,2.0,240.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
50%,370.000000,162.162162,162.601626,2.0,240.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
75%,400.000000,168.539326,168.539326,2.0,240.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
max,898.000000,172.910663,170.940171,2.0,240.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


,rr_interval,bpm_raw,bpm_clean,intensity,artifacts,person_id,sex,age,diet,caffeine,sleep_q,sleep_h,wellbeeing,cold_hist,asthma,smoker,fitness_level,study_modus,bmi,rpm
count,431.000000,431.000000,431.000000,431.0,431.0,431.0,431.0,431.0,431.0,431.0,431.0,431.0,431.0,431.0,431.0,431.0,431.0,431.0,431.0,431.0
mean,351.638051,173.522031,174.610292,3.0,288.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
std,52.810665,19.728399,17.689261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,313.000000,92.307692,122.199593,3.0,288.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25%,322.000000,169.015426,171.428571,3.0,288.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
50%,327.000000,183.486239,183.486239,3.0,288.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
75%,355.000000,186.335404,186.335404,3.0,288.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
max,650.000000,191.693291,188.679245,3.0,288.0,2.0,0.0,22.0,0.0,0.0,0.0,0.5,11.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


,rr_interval,bpm_raw,bpm_clean,intensity,artifacts,person_id,sex,age,diet,caffeine,sleep_q,sleep_h,wellbeeing,cold_hist,asthma,smoker,fitness_level,study_modus,bmi,rpm
count,362.000000,362.000000,362.000000,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0
mean,423.754144,144.996898,146.072352,1.0,231.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
std,81.655128,18.730793,16.549029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,364.000000,50.547599,89.020772,1.0,231.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
25%,377.000000,137.693768,138.328668,1.0,231.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
50%,396.000000,151.515152,152.671756,1.0,231.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
75%,435.750000,159.151194,159.574468,1.0,231.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
max,1187.000000,164.835165,163.043478,1.0,231.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0


,rr_interval,bpm_raw,bpm_clean,intensity,artifacts,person_id,sex,age,diet,caffeine,sleep_q,sleep_h,wellbeeing,cold_hist,asthma,smoker,fitness_level,study_modus,bmi,rpm
count,409.000000,409.000000,409.000000,409.0,409.0,409.0,409.0,409.0,409.0,409.0,409.0,409.0,409.0,409.0,409.0,409.0,409.0,409.0,409.0,409.0
mean,372.779951,164.267453,164.286973,2.0,241.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
std,60.248712,20.868993,20.833004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,326.000000,89.418778,89.418778,2.0,241.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
25%,335.000000,151.515152,151.515152,2.0,241.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
50%,344.000000,174.418605,173.913043,2.0,241.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
75%,396.000000,179.104478,178.571429,2.0,241.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
max,671.000000,184.049080,182.926829,2.0,241.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0


,rr_interval,bpm_raw,bpm_clean,intensity,artifacts,person_id,sex,age,diet,caffeine,sleep_q,sleep_h,wellbeeing,cold_hist,asthma,smoker,fitness_level,study_modus,bmi,rpm
count,438.000000,438.000000,438.000000,438.0,438.0,438.0,438.0,438.0,438.0,438.0,438.0,438.0,438.0,438.0,438.0,438.0,438.0,438.0,438.0,438.0
mean,351.390411,174.876403,175.092972,3.0,291.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
std,72.451234,21.313844,20.866397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,309.000000,65.359477,65.359477,3.0,291.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
25%,319.000000,169.611562,170.940171,3.0,291.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
50%,326.000000,184.049080,184.615385,3.0,291.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
75%,353.750000,188.087774,188.087774,3.0,291.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
max,918.000000,194.174757,191.693291,3.0,291.0,1.0,1.0,26.0,0.0,1.0,2.0,7.0,20.0,0.0,0.0,1.0,0.0,1.0,21.0,0.0
